In [1]:
# Download the dataset
# You may choose where to download the data.

# Google Drive
#!gdown --id '1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy' --output food-11.zip

# Dropbox
# !wget https://www.dropbox.com/s/m9q6273jl3djall/food-11.zip -O food-11.zip

# MEGA
# !sudo apt install megatools
# !megadl "https://mega.nz/#!zt1TTIhK!ZuMbg5ZjGWzWX1I6nEUbfjMZgCmAgeqJlwDkqdIryfg"

# Unzip the dataset.
# This may take some time.
#!unzip -q food-11.zip

In [2]:
#import pytorch
import torch

# torch.backends.cudnn: set CNN algorithmtorch.backends.cudnn
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# get the current available device ('cpu' or 'cuda')
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'
device = get_device()
print(torch.cuda.is_available())
#set random variable
import numpy as np
myseed = 1
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

True


In [3]:
# import torch
# import torch.nn as nn
# m = nn.Conv2d(1, 1, 2 , stride=2, padding=(9,9),bias = False)
# test = torch.tensor([[[1.0,2.0,3.0],[4.0,5.0,6.0],[7.0,8.0,9.0]]])
# print(test)
# print(m(test))

In [9]:
# from PIL import Image
# with Image.open("C:/Users/user/Pictures\Screenshots/資格證.jpg") as im:
#     im.show()

In [3]:
# import torch
# x = torch.FloatTensor([[2,7,1],[3,4,9],[5,6,8]])
# y = x[x> 2]
# print(x,y)
# print(x.argmax(dim = 0))
# print(x.argmax(dim = 1))
# print(x.argmax(dim = -1))

In [35]:
import torchvision.transforms as transforms
from torchvision.datasets import DatasetFolder
from PIL import Image
def get_tfm(mode):
    if mode in ["val","test"]:
      tfm = transforms.Compose([
      transforms.Resize((128, 128)),
      transforms.ToTensor(),
      ])
    elif mode in ["train_unlabeled","train"]:
       tfm = transforms.Compose([
      transforms.Resize((128, 128)),
      transforms.ToTensor(),
      ])
    return tfm
def food_f(mode):
    dataset = DatasetFolder(data_info[mode]["path"], loader=lambda x: Image.open(x), extensions="jpg", transform=get_tfm(mode))
    print('Size of {} data: {}'.format(mode,len(list(dataset))))
    return dataset

#create a dict of functions and path w.r.t. different mode
data_info = {
    "train_origin":{"path":"./food-11/training/labeled"},
    "train":{"path":"./food-11/training/unlabeled"},
    "val":{"path":"./food-11/validation"},
    "test":{"path":"./food-11/testing"}
}

In [32]:

tfm = transforms.Compose([
      transforms.Resize((128, 128)),
      transforms.ToTensor(),
      ])
print(type(tfm))

<class 'torchvision.transforms.transforms.Compose'>


In [36]:
x = 500# x = int(max(prep))
print(list(food_f("val"))[x][1])

Size of val data: 660
8


In [37]:
h_paras = {
    # maximum number of epochs
    'n_epochs': 200,
    # mini-batch size for dataloader
    'batch_size': 128,
    # optimization algorithm (optimizer in torch.optim)
    'optimizer': 'Adam',
    # hyper-parameters for the optimizer (depends on which optimizer you are using)
    'optim_hparas': {
        # learning rate of Adam
        'lr': 0.001,
    },
    'early_stop': 5,
    # your model will be saved here
    'save_path': './model.pth'
}

In [38]:
from torch.utils.data import DataLoader

# Construct data loaders.
train_loader = DataLoader(food_f("train"), batch_size=h_paras["batch_size"], shuffle=True, num_workers=8, pin_memory=True)
valid_loader = DataLoader(food_f("val"), batch_size=h_paras["batch_size"], shuffle=True, num_workers=8, pin_memory=True)
test_loader = DataLoader(food_f("test"), batch_size=h_paras["batch_size"], shuffle=False)

Size of train data: 6786


d:\Anaconda\envs\ML2021_python_3_11_5\Lib\site-packages\torch\utils\data\dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Size of val data: 660
Size of test data: 3347


In [39]:
from torch import nn
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # The arguments for commonly used modules:
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)

        # input image size: [3, 128, 128]
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(4, 4, 0),
        )
        self.flat_layer = nn.Flatten()
        self.fc_layers = nn.Sequential(
            nn.Linear(256 * 8 * 8, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 11)
        )

    def forward(self, x):
        # input (x): [batch_size, 3, 128, 128]
        # output: [batch_size, 11]

        # Extract features by convolutional layers.
        x = self.cnn_layers(x)

        # The extracted feature map must be flatten before going to fully-connected layers.
        x = self.flat_layer(x)

        # The features are transformed by fully-connected layers to obtain the final logits.
        x = self.fc_layers(x)
        return x

In [ ]:
def val(val_set,model_d,device):
    # set model to evalutation mode
    model_d.eval()
    total_correct_number = 0
    total_loss = 0
    # iterate through the dataloader
    for data , label in val_set:
    # move data to device (cpu/cuda)
      data_d, label_d = data.to(device), label.to(device)
      # disable gradient calculation
      with torch.no_grad():
        # forward pass (compute output)
        pred = model_d(data_d)
        # get the index of the class with the highest probability
        max_prob_values, max_prob_indexs = torch.max(pred, dim = 1)
        total_correct_number += (max_prob_indexs.cpu() == label_d.cpu()).sum().item()
        # compute loss
        mse_loss = model_d.cal_loss(pred, label_d)
      # accumulate loss
      batch_size = len(data_d)
      total_loss += mse_loss.detach().cpu().item() * batch_size

    # compute averaged loss
    totol_size = len(val_set.dataset)
    acc = total_correct_number/totol_size
    avg_loss =  total_loss/totol_size
    return acc, avg_loss

In [40]:
from tqdm.auto import tqdm
def get_pseudo_labels(dataset, model, threshold=0.65):
    # This functions generates pseudo-labels of a dataset using given model.
    # It returns an instance of DatasetFolder containing images whose prediction confidences exceed a given threshold.
    # You are NOT allowed to use any models trained on external data for pseudo-labeling.

    # Construct a data loader.
    train_unlabeled_loader = \
    DataLoader(food_f("train_unlabeled"), batch_size=h_paras["batch_size"], shuffle=True, num_workers=8, pin_memory=True)

    # Make sure the model is in eval mode.
    model.eval()
    # Define softmax function.
    softmax = nn.Softmax(dim=-1)

    # Iterate over the dataset by batches.
    for batch in tqdm(train_unlabeled_loader):
        img, label = batch

        # Forward the data
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(img.to(device))

        # Obtain the probability distributions by applying softmax on logits.
        predicts = softmax(logits)
        for  index , probs in enumerate(predicts):
            max_index , max_prob = probs.max()
            if max_prob > threshold:
                (logits[index],max_index)


        # Filter the data and construct a new dataset.
        



    # # Turn off the eval mode.
    model.train()
    return dataset

d:\Anaconda\envs\ML2021_python_3_11_5\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
